### Bayesian Hierarchical Ratings
https://penaltyblog.readthedocs.io/en/latest/ratings/massey_ratings.html

https://www.pro-football-reference.com/years/2022/games.htm


In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
import os
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import penaltyblog as pb

In [3]:
import nfl_data_py as nfl

In [4]:
try:
    import nfl_data_py as nfl
except:
    ! pip install nfl_data_py

In [5]:
semana = 7
temporada = 2022
normalizacion = 1
MAX = 10
df = nfl.import_schedules([temporada])
df = df[df['week']<semana]

In [6]:
df = df.rename(columns={'away_team':'AwayTeam',
                                   'home_team':'HomeTeam',
                       'away_score':'FTHG',
                       'home_score':'FTAG'})


In [7]:
df[['FTHG','FTAG']]

,FTHG,FTAG
6137,31.0,10.0
6138,27.0,26.0
6139,26.0,24.0
6140,10.0,19.0
6141,23.0,20.0
...,...,...
6226,10.0,24.0
6227,9.0,19.0
6228,24.0,20.0
6229,17.0,26.0


In [8]:
df[df['AwayTeam']=='BUF']

,game_id,season,game_type,week,gameday,weekday,gametime,AwayTeam,FTHG,HomeTeam,...,wind,away_qb_id,home_qb_id,away_qb_name,home_qb_name,away_coach,home_coach,referee,stadium_id,stadium
6137,2022_01_BUF_LA,2022,REG,1,2022-09-08,Thursday,20:20,BUF,31.0,LA,...,NaN,00-0034857,00-0026498,Josh Allen,Matthew Stafford,Sean McDermott,Sean McVay,Carl Cheffers,LAX01,SoFi Stadium
6173,2022_03_BUF_MIA,2022,REG,3,2022-09-25,Sunday,13:00,BUF,19.0,MIA,...,NaN,00-0034857,00-0036212,Josh Allen,Tua Tagovailoa,Sean McDermott,Mike McDaniel,Alex Kemp,MIA00,Hard Rock Stadium
6188,2022_04_BUF_BAL,2022,REG,4,2022-10-02,Sunday,13:00,BUF,23.0,BAL,...,NaN,00-0034857,00-0034796,Josh Allen,Lamar Jackson,Sean McDermott,John Harbaugh,Jerome Boger,BAL00,M&T Bank Stadium
6228,2022_06_BUF_KC,2022,REG,6,2022-10-16,Sunday,16:25,BUF,24.0,KC,...,NaN,00-0034857,00-0033873,Josh Allen,Patrick Mahomes,Sean McDermott,Andy Reid,Brad Allen,KAN00,GEHA Field at Arrowhead Stadium


In [9]:
if normalizacion == 1:
    df['FTHG'] = (df['FTHG'] - df['FTHG'].min()) / ( df['FTHG'].max() - df['FTHG'].min()) * MAX
    df['FTAG'] = (df['FTAG'] - df['FTAG'].min()) / ( df['FTAG'].max() - df['FTAG'].min()) * MAX

In [10]:
df[["FTHG", "FTAG"]].mean()

FTHG    4.392730
FTAG    4.566869
dtype: float64

In [11]:
df[['FTHG','FTAG']]

,FTHG,FTAG
6137,6.458333,1.666667
6138,5.625000,5.476190
6139,5.416667,5.000000
6140,2.083333,3.809524
6141,4.791667,4.047619
...,...,...
6226,2.083333,5.000000
6227,1.875000,3.809524
6228,5.000000,4.047619
6229,3.541667,5.476190


In [18]:
pr = nfl.import_schedules([temporada])
pr = pr[pr['week']==semana ]
juegos = pr['week'].count()

In [19]:
pr = pr.rename(columns={'away_team':'AwayTeam',
                                   'home_team':'HomeTeam',
                       'away_score':'FTHG',
                       'home_score':'FTAG'})

In [20]:
pois = pb.models.BayesianBivariateGoalModel(df["FTHG"], df["FTAG"], df["HomeTeam"], df["AwayTeam"])
pois.fit()
print('Bayesian Hierarchical')
for i in range(0,juegos):
    EL = pr.iloc[i]['HomeTeam']
    EV = pr.iloc[i]['AwayTeam']
    
    probs = pois.predict(EL, EV)
    print(f'Equipo Local {EL} vs Equipo Visitante {EV}')
    print('Local               - Empate -             Visitante')
    print(probs.home_draw_away)

AttributeError: module 'penaltyblog.models' has no attribute 'BayesianBivariateGoalModel'